# This is an example of opening multiple .root file using uproot4, dask and displaying data into a single aggregated histogram

## 1. Importing all the necessary packages

In [7]:
import sys
sys.path.insert(0, '..')
import uproot4
import python.file_manager as fm
import ROOT
import numpy as np
import fsspec
import distributed
import awkward as ak
import dask_awkward as dak
import hist
import dask_histogram as dh
import boost_histogram as bh
import uproot
import matplotlib.pyplot as plt
import dask.array as da
import dask
import hist.dask as dah
import time
print("Loaded")

Loaded


## 2. Opening a distributed client for Dask

In [3]:
client = distributed.Client()
client

/data/gsvedas/.virtualenvs/gintautas_fast/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44515 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:44515/status,
Dashboard: http://127.0.0.1:44515/status,Workers: 8
Total threads: 48,Total memory: 62.29 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38483,Workers: 8
Dashboard: http://127.0.0.1:44515/status,Total threads: 48
Started: Just now,Total memory: 62.29 GiB
Comm: tcp://127.0.0.1:35875,Total threads: 6
Dashboard: http://127.0.0.1:39661/status,Memory: 7.79 GiB
Nanny: tcp://127.0.0.1:39073,


# 3. Opening .root files using Dask and Uproot

### 3.1 Opening multiple .ROOT files using Dask

In [ ]:
dask_multiple_files = uproot4.dask({'root://eoscms.cern.ch//eos/cms/store/cmst3/group/l1tr/cerminar/l1teg/fp_ntuples/DoubleElectron_FlatPt-1To100_PU200/FP/v100D/perfNano_8830991_0.root': 'Events',
                   'root://eoscms.cern.ch//eos/cms/store/cmst3/group/l1tr/cerminar/l1teg/fp_ntuples/DoubleElectron_FlatPt-1To100_PU200/FP/v100D/perfNano_8830991_1.root': 'Events'
                   })

### 3.2 Opening single .ROOT file using DASK

In [ ]:
dask_single_file = uproot4.dask({'root://eoscms.cern.ch//eos/cms/store/cmst3/group/l1tr/cerminar/l1teg/fp_ntuples/DoubleElectron_FlatPt-1To100_PU200/FP/v100D/perfNano_8830991_0.root': 'Events'})

In [ ]:
dask_single_third = up.dask({'root://eoscms.cern.ch//eos/cms/store/cmst3/group/l1tr/cerminar/l1teg/fp_ntuples/DoubleElectron_FlatPt-1To100_PU200/FP/v102D/perfNano_9026841_0.root': 'Events'}, library='np')

# 4. Experimenting with .persist() option

In [ ]:
import dask.array as da

# Create a Dask array
x = da.random.random((1000, 1000), chunks=(100, 100))

# Perform some operations on the array
y = (x + x.T).sum()

# Trigger computation and persist result in memory
y_persisted = y.persist()

In [ ]:
y_persisted.compute()

# 5. Opening MULTIPLE files using uproot.open

In [5]:
file_first =  uproot4.open('root://eoscms.cern.ch//eos/cms/store/cmst3/group/l1tr/cerminar/l1teg/fp_ntuples/DoubleElectron_FlatPt-1To100_PU200/FP/v100D/perfNano_8830991_0.root')
tree_first = file_first['Events']

file_second =  uproot4.open('root://eoscms.cern.ch//eos/cms/store/cmst3/group/l1tr/cerminar/l1teg/fp_ntuples/DoubleElectron_FlatPt-1To100_PU200/FP/v100D/perfNano_8830991_1.root')
tree_second = file_second['Events']

arr_first = tree_first.arrays(['nTkEleL2'], library='ak')
arr_second = tree_second.arrays(['nTkEleL2'], library='ak')

arr_first_second_final = ak.concatenate([arr_first, arr_second])

In [ ]:
file_third =  up.open('root://eoscms.cern.ch//eos/cms/store/cmst3/group/l1tr/cerminar/l1teg/fp_ntuples/DoubleElectron_FlatPt-1To100_PU200/FP/v100D/perfNano_8830991_0.root')
tree_third = file_third['Events']

## 5.1 Checking the input of opened files

In [ ]:
print(dask_multiple_files['nTkEleL2'])

In [ ]:
print(dask_single_file['nTkEleL2'])

In [ ]:
print(arr[0])

## 5.2 Functions for set-up of the charts

In [ ]:
def plot_chart(data):
    # add the axes, finalize with storage
    h = (
        dah.Hist.new.Reg(50, -5, 5, name="S", label="s [units]", flow=False)
        .Reg(50, -5, 5, name="W", label="w [units]", flow=False)
        .Double()
    )

    s_data = data
    w_data = data
        
    # delayed fill
    h.fill(W=w_data, S=s_data)
    
    # auto-plot
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    
    h.project("W").plot(ax=axs[0])
    h.project("W", "S").plot(ax=axs[1])
    plt.show()
    
    return h

In [ ]:
def finalize_dask_chart(histogram):
    # render in-memory histogram
    print("COMPUTING THIS: ", histogram)
    histogram = histogram.compute()

    import matplotlib.pyplot as plt

    # auto-plot
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))

    histogram.project("W").plot(ax=axs[0])
    histogram.project("W", "S").plot(ax=axs[1])
    plt.show()

## 5.3 Dask charts: timing of the dask charts

In [ ]:
start = time.time()

dask_chart_single = plot_chart(dask_single_file['nTkEleL2'])
finalize_dask_chart(dask_chart_single)

end = time.time()
print(end - start)

## 5.4  uproot: plot a chart from a single .ROOT file

In [ ]:
uproot_chart_single = plot_chart(tree['nTkEleL2'])

## 5.5 uproot charts: plot a chart from multiple .ROOT files that are stiched via ak.concatenate

In [ ]:
uproot_multiple_files = plot_chart(arr_first_second_final)

# 5.6 Compare Dask with single ROOT file VS uproot4 single ROOT file

### 5.6.1 uproot

In [ ]:
bin_contents_uproot = uproot_chart_single.values()

print("LENGTH OF THE BINS: ",  bin_contents_uproot.size)

for i, content in enumerate(bin_contents_uproot):
    print(f"Bin {i}: {content}")

### 5.6.2 dask

In [ ]:
bin_contents_dask_single = dask_chart_single.values()

print("LENGTH OF THE BINS: ",  bin_contents_dask_single.size)

for i, content in enumerate(bin_contents_dask_single):
    print(f"Bin {i}: {content}")

# 6 Dask charts: opening MULTIPLE ROOT files via multi stage approach

### plot_chart initializes the initial chart

In [ ]:
dask_chart_multiple = plot_chart(dask_multiple_files['nTkEleL2'])

### finalize_dask_chart calls .compute() on the data and plots the chart

In [ ]:
finalize_dask_chart(dask_chart_multiple)

## 6.1 Compare Dask with MULTIPLE ROOT file VS uproot4 MULTIPLE ROOT file

#### 6.1.1 uproot

In [ ]:
bin_contents_uproot_multiple = uproot_chart_multiple.values()

print("LENGTH OF THE BINS: ",  bin_contents_uproot_multiple.size)

for i, content in enumerate(bin_contents_uproot_multiple):
    print(f"Bin {i}: {content}")

#### 6.1.2 dask

In [ ]:
bin_contents_dask_multiple = dask_chart_multiple.values()

print("LENGTH OF THE BINS: ",  bin_contents_dask_multiple.size)

for i, content in enumerate(bin_contents_dask_multiple):
    print(f"Bin {i}: {content}")